# Week 2 - Preprocessing, part 2

# 1. Lesson: None

# 2. Weekly graph question

The Storytelling With Data book mentions planning on a "Who, What, and How" for your data story.  Write down a possible Who, What, and How for your data, using the ideas in the book.

Who is this data story for? For sports analytics nerds like myself, who find interest in data patterns and data anomalies. There is also an audience who seeks to better understand the lifecycle of a tennis player.\
What does my audience need to know or do? For this story, its good to have a general background of tennis, with knowledge of how long the seasons are and the types of surfaces that are played. Tennis has a long history that dates back more that a couple centuries, however the focus of this story will be over the past 4 decades. I want to illustrate to the audience, the statistics of what it takes to become a professional tennis player, the evolution of styles over the past couple of decades, and general patterns & anomalies in certain players.\
How do I want to display this information to the viewer, how can I use the data to help make my point? Need to do some EDA but will come back to this

# 3. Homework - work with your own data

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import glob

In [3]:
# Import necessary libraries
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect("database.sqlite")

# Create a cursor object
cursor = conn.cursor()

# Fetch all tables names in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Display table names
print("Tables in the database:")
for table in tables:
    print(table[0])

Tables in the database:
matches
players
rankings


In [4]:
# query on the matches table
query_match = "SELECT * FROM matches where tourney_date > 19980101.0;"

df_match = pd.read_sql_query(query_match, conn)

df_match.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,loser2_ioc,loser2_age,winner1_rank,winner1_rank_points,winner2_rank,winner2_rank_points,loser1_rank,loser1_rank_points,loser2_rank,loser2_rank_points
0,2018-M020,Brisbane,Hard,32,A,20180101.0,271,105992.0,None,None,...,None,None,None,None,None,None,None,None,None,None
1,2018-M020,Brisbane,Hard,32,A,20180101.0,272,111577.0,None,None,...,None,None,None,None,None,None,None,None,None,None
2,2018-M020,Brisbane,Hard,32,A,20180101.0,273,104797.0,None,None,...,None,None,None,None,None,None,None,None,None,None
3,2018-M020,Brisbane,Hard,32,A,20180101.0,275,200282.0,None,WC,...,None,None,None,None,None,None,None,None,None,None
4,2018-M020,Brisbane,Hard,32,A,20180101.0,276,111581.0,None,Q,...,None,None,None,None,None,None,None,None,None,None


In [5]:
# query on the ranking table
# filtering the match data to get the top 100 ranked players
query_ranking = "SELECT * FROM rankings where ranking_date > 20000101 and rank < 100;"

df_ranking = pd.read_sql_query(query_ranking, conn)

df_ranking.head()

,ranking_date,rank,player,points
0,20000110,1,101736,4135.0
1,20000110,2,102338,2915.0
2,20000110,3,101948,2419.0
3,20000110,4,103017,2184.0
4,20000110,5,102856,2169.0


In [6]:
# query the players table
query_players = "SELECT * FROM players;"

df_players = pd.read_sql_query(query_players, conn)

df_players.head()

,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id
0,100001,Gardnar,Mulloy,R,19131122.0,USA,185.0,Q54544
1,100002,Pancho,Segura,R,19210620.0,ECU,168.0,Q54581
2,100003,Frank,Sedgman,R,19271002.0,AUS,180.0,Q962049
3,100004,Giuseppe,Merlo,R,19271011.0,ITA,NaN,Q1258752
4,100005,Richard,Gonzalez,R,19280509.0,USA,188.0,Q53554


In [7]:
# Close the cursor and connection
cursor.close()
conn.close()

# Summarizing the Datasets

In [8]:
df_match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 664948 entries, 0 to 664947
Data columns (total 81 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   tourney_id           664948 non-null  object 
 1   tourney_name         664948 non-null  object 
 2   surface              663195 non-null  object 
 3   draw_size            664948 non-null  object 
 4   tourney_level        664948 non-null  object 
 5   tourney_date         664948 non-null  float64
 6   match_num            664948 non-null  object 
 7   winner_id            664948 non-null  float64
 8   winner_seed          299623 non-null  object 
 9   winner_entry         123957 non-null  object 
 10  winner_name          664948 non-null  object 
 11  winner_hand          664848 non-null  object 
 12  winner_ht            379262 non-null  float64
 13  winner_ioc           664830 non-null  object 
 14  winner_age           662849 non-null  float64
 15  loser_id         

In [9]:
df_match.describe()

,tourney_date,winner_id,winner_ht,winner_age,loser_id,loser_ht,loser_age,minutes,w_ace,w_df,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
count,6.649480e+05,664948.000000,379262.000000,662849.000000,664948.000000,297713.000000,658462.000000,176993.000000,182361.000000,182361.000000,...,182361.000000,182361.000000,182361.000000,182360.000000,182362.000000,182362.000000,634234.000000,633149.000000,584672.000000,583422.000000
mean,2.011496e+07,117787.040293,184.700843,23.997762,120585.088345,184.659948,23.552829,100.302323,5.678945,2.609884,...,45.290095,29.535114,13.641996,11.375285,4.591395,8.454969,491.241085,303.285446,616.484901,210.964107
std,7.374771e+04,30848.113375,6.551211,3.897129,32861.882401,6.543601,4.110486,39.061500,4.746302,2.224468,...,16.904146,12.714757,6.519451,3.631714,3.103207,3.809469,377.750723,847.126041,466.192465,514.020911
min,1.998010e+07,100236.000000,145.000000,14.000000,100236.000000,145.000000,14.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,2.005102e+07,103885.000000,180.000000,21.000000,104026.000000,180.000000,20.400000,73.000000,2.000000,1.000000,...,33.000000,20.000000,9.000000,9.000000,2.000000,6.000000,203.000000,28.000000,233.000000,12.000000
50%,2.012061e+07,105084.000000,185.000000,23.600000,105409.000000,185.000000,23.000000,93.000000,5.000000,2.000000,...,43.000000,28.000000,13.000000,10.000000,4.000000,8.000000,416.000000,84.000000,520.000000,54.000000
75%,2.017102e+07,108761.000000,188.000000,26.400000,120922.000000,188.000000,26.100000,122.000000,8.000000,4.000000,...,55.000000,37.000000,17.000000,14.000000,6.000000,11.000000,680.000000,237.000000,906.000000,200.000000
max,2.024052e+07,212915.000000,211.000000,101.500000,212917.000000,211.000000,101.500000,4756.000000,113.000000,26.000000,...,328.000000,284.000000,101.000000,91.000000,27.000000,38.000000,2257.000000,16950.000000,2267.000000,16950.000000


In [10]:
df_ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115830 entries, 0 to 115829
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ranking_date  115830 non-null  int64  
 1   rank          115830 non-null  int64  
 2   player        115830 non-null  int64  
 3   points        115830 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 3.5 MB


In [11]:
df_ranking.describe()

,ranking_date,rank,player,points
count,1.158300e+05,115830.000000,115830.000000,115830.000000
mean,2.011229e+07,50.000000,108654.284028,1346.711301
std,6.993747e+04,28.577504,18069.961758,1492.389061
min,2.000011e+07,1.000000,100644.000000,372.000000
25%,2.005082e+07,25.000000,103428.000000,647.000000
50%,2.011041e+07,50.000000,104312.000000,879.000000
75%,2.017081e+07,75.000000,105238.000000,1380.000000
max,2.024053e+07,99.000000,210506.000000,16950.000000


In [12]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65019 entries, 0 to 65018
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   player_id    65019 non-null  int64  
 1   name_first   64134 non-null  object 
 2   name_last    64971 non-null  object 
 3   hand         65002 non-null  object 
 4   dob          46447 non-null  float64
 5   ioc          64343 non-null  object 
 6   height       2819 non-null   float64
 7   wikidata_id  4894 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 4.0+ MB


In [13]:
df_players.describe()

,player_id,dob,height
count,65019.000000,4.644700e+04,2819.000000
mean,142051.658223,1.981089e+07,183.748138
std,34841.271088,2.007141e+05,6.547397
min,100001.000000,1.834000e+07,145.000000
25%,116270.500000,1.976093e+07,180.000000
50%,132525.000000,1.985061e+07,183.000000
75%,148780.500000,1.993063e+07,188.000000
max,212917.000000,2.008011e+07,211.000000


In [14]:
# Looking for duplicate values in the tables
match_dupes = df_match.duplicated().sum()
rank_dupes = df_ranking.duplicated().sum()
players_dupes = df_players.duplicated().sum()

print("Duplicate values in the matches table: ", match_dupes)
print("Duplicate values in the rankings table: ", rank_dupes)
print("Duplicate values in the players table: ", players_dupes)

Duplicate values in the matches table:  0
Duplicate values in the rankings table:  0
Duplicate values in the players table:  0


In [15]:
# Checking for duplicate matches in the matches table
df_match["match_id"] = df_match['tourney_id'] + "-" + df_match['match_num'].astype(str)
match_id_dupes = df_match["match_id"].duplicated()
print("Duplicate match IDs in the matches table: ", match_id_dupes.sum())

Duplicate match IDs in the matches table:  2016


There are duplicates in the df_match table on match_id. This is supposed to be a unique element for each match, however sopme of the records have not been entered in correctly (i.e. both players in the match have been listed as winners in separate records, in the same instance the wrong player was listed who did not participate in the match). Duplicate values are allowed for columns that denote a players dominant hand (only 1 of two options) or the players seed in the tournament. 

In [25]:
# What are the mean, median, and mode of the match columns
df_match_mean = df_match.mean(numeric_only=True)
df_match_median = df_match.median(numeric_only=True)
df_match_mode = df_match.mode(numeric_only=True)

match_stats = pd.concat([df_match_mean, df_match_median, df_match_mode], axis=1)

match_stats

,0,1,tourney_date,winner_id,winner_ht,winner_age,loser_id,loser_ht,loser_age,minutes,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
tourney_date,2.011496e+07,20120611.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
winner_id,1.177870e+05,105084.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
winner_ht,1.847008e+02,185.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
winner_age,2.399776e+01,23.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loser_id,1.205851e+05,105409.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loser_ht,1.846599e+02,185.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loser_age,2.355283e+01,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
minutes,1.003023e+02,93.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
w_ace,5.678945e+00,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
w_df,2.609884e+00,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


This week, you will do the same types of exercises as last week, but you should use your own datasets that you found last semester.

### Here are some types of analysis you can do  Use Google, documentation, and ChatGPT to help you:

- Summarize the datasets using info() and describe()

- Are there any duplicate rows?

- Are there any duplicate values in a given column (when this would be inappropriate?)

- What are the mean, median, and mode of each column?

- Are there any missing or null values?

    - Do you want to fill in the missing value with a mean value?  A value of your choice?  Remove that row?

- Identify any other inconsistent data (e.g. someone seems to be taking an action before they are born.)

- Encode any categorical variables (e.g. with one-hot encoding.)

### Conclusions:

- Are the data usable?  If not, find some new data!

- Do you need to modify or correct the data in some way?

- Is there any class imbalance?  (Categories that have many more items than other categories).

# 4. Storytelling With Data graph

Just like last week: choose any graph in the Introduction of Storytelling With Data. Use matplotlib to reproduce it in a rough way. I don't expect you to spend an enormous amount of time on this; I understand that you likely will not have time to re-create every feature of the graph. However, if you're excited about learning to use matplotlib, this is a good way to do that. You don't have to duplicate the exact values on the graph; just the same rough shape will be enough.  If you don't feel comfortable using matplotlib yet, do the best you can and write down what you tried or what Google searches you did to find the answers.